In [1]:
import numpy as np

In [17]:
class BinaryTree():
    '''
    Train Data:
    X in shape (m, n)
    m = num_samples, n = num_features,
    Only one_hot values, no multiclass or regression
    Features Must Not be single valued: e.g 
    >>> X = np.array([[0],[0],[0],[0],[0],[0],[0]])
    >>> y = np.array([ 0 , 1 , 0 , 0 , 1 , 1 , 0 ])
    '''
    class Node():

        def __init__(self, depth=0, parent=None):
            self.depth = depth
            self.parent = parent
            self.h = 1

    class Decisive(Node):

        def __init__(self, feature, depth=0, parent=None):
            self.feature = feature
            self.children = {}
            self.received_features = None
            super().__init__(depth=depth, parent=parent)
            


    class Root(Decisive):
        
        def __init__(self,feature):
            super().__init__(feature=feature, )

    class Decision(Decisive):
        
        def __init__(self, feature, parent, depth=0, ):
            super().__init__(feature=feature, depth=depth,  parent=parent)

    class Leaf(Node):
        
        def __init__(self, parent, confidence=1, depth=1,output=1):
            self.confidence = confidence
            self.output = output
            super().__init__(depth=depth, parent=parent)
            

    def __init__(self, threshold=1, depth=3):
        self.threshold = np.clip(threshold, 0, 1)
        self.max_depth=np.maximum(1,depth)
        self.root  = None
       


    def H(self, x) -> int:
        if x == 0 or x ==1:
            return 0
        return -x*np.log2(x) - (1-x) * np.log2(1-x)
    
    def split(self, X, feature):
        '''
        Returns
        ======================
        feature=True, feature=False
        '''
        return X[X[:,feature]==1], X[X[:,feature]==0]
        
    def information_gain(self, X, y, feature):
        '''
        Return 
        =================
        Information gain, whether to flip positive and negative
        '''
        if y.size == 0:
            raise KeyError('Y got size of zero')
        if y.mean() == 0 or y.mean() == 1:
            raise ValueError('Completely Pure Node recieved')
        X = np.concatenate([X,y.reshape(-1,1)], axis=-1)

        tot = X.shape[0]
        
        positive, negative = self.split(X,feature)
        
        if positive.size == 0 or negative.size== 0:
            return 0

        w1, w0 = positive.shape[0]/tot ,negative.shape[0]/tot
        
        p1, p0 =  (positive[:,-1]).mean(), (negative[:,-1]).mean()

        return self.H(np.mean(y)) - (w1 * self.H(p1) + w0 * self.H(p0))

    def get_best(self, X :np.ndarray,y :np.ndarray) -> int:
        '''
        Get the best feature to split on
        
        Returns :
        _________________________________
        
        best_feature_column
        '''
        best = 0
        max = 0
        for i in range(X.shape[-1]):
            x= self.information_gain(X,y,i)
            if best < x:
                best = x
                max = i
        return max


    def train(self, X, y, node : Decisive= None, verbose=False):
        
        end = False
        # print('Current data\n', X, '\n', y)
        removes = []
        for f in range(X.shape[-1]):
            if X[:,f].mean()==0 or X[:,f].mean()==1:
                removes.append(f)
        if len(removes):
            X=np.delete(X, removes, axis=-1)
            node.received_features = np.delete(node.received_features, removes, axis=0)

            # print('-----------------------------------')
            # print('Current data\n', X, '\n', y)
        if node is not None:
            prob = y.mean()
            p = np.maximum(prob, 1-prob)
            if verbose:
                print('Confidence', f'{p:4f}', 'Num Samples at Node :' , len(y))
            node.h = self.H(p)
            if p >= self.threshold:
                end = True
                if verbose:
                    print('Threshold Met ')
            elif X.size == 0:
                end=True
                if verbose:
                    print('End of Features')
            elif node.depth == self.max_depth:
                end=True
                if verbose:
                    print('Maximum Depth Reached')
            if end:
                node : self.Leaf = self.Leaf(parent=node.parent, confidence=p, depth=node.depth)
                node.output = int(prob>=0.5)
                node.h = self.H(p)
                return node
            if verbose:
                print('Threshold Not met ')

        if node is None:
            if verbose:
                print('Initializing Root')
            node = self.Root(feature=None)
            self.root =  node
            node.h = self.H(y.mean())
            node.received_features = np.arange(X.shape[-1]) 
        
        if verbose:
            print('Getting Best feature')
        best_feature = self.get_best(X, y)
        node.feature = node.received_features[best_feature]
        if verbose:
            print('Retrieved best feature : ',node.feature)

        p,n = self.split(X = np.concatenate([X,y.reshape(-1,1)], axis=-1), feature=best_feature)
        positive,  pos_y, negative, neg_y = p[:,:-1], p[:,-1], n[:,:-1], n[:,-1]
        # print('Positive split : \n', positive,'\n', pos_y)
        # print('Negative split : \n', negative,'\n', neg_y)
   
        for i, (X,y) in enumerate(([negative, neg_y], [positive, pos_y])):
            if verbose:
                print(f'Setting Child {i} of Depth' ,node.depth+1)
            n = self.Decisive(feature=None,depth=node.depth+1, parent=node)
            n.received_features = np.delete(node.received_features, best_feature, axis=-1)
            node.children[i] = self.train(X=np.delete(X, best_feature, axis=-1), y=y,node=n, verbose=verbose)
        return node
    
    def pretty_print(self, node=None):

        if self.root is None:
            raise LookupError('Unntrained Decision Tree')
        if node is None:
            node = self.root
        print('='+'======'*node.depth+'>', 'Depth :', node.depth,'    |||     Entropy : ', f'{node.h:2f}',  end='')
        if type(node) == self.Leaf:
            print('   |||    Confidence', f'{node.confidence:2f}', '   |||    Output', node.output)
            return
        else:
            print('   |||    Feature', node.feature, '|||   Num Features: ', len(node.received_features))
        for child in node.children.values():
            self.pretty_print(child)
        
    def predict(self, X):
        '''
        Must receive shape (m,n)
        m = num_fsamples - (1>=)
        n = num_features - (train_data_shape)
        '''
        preds = []
        for sample in X:
            node : self.Decisive = self.root
            while type(node)!=self.Leaf:

                node = node.children[sample[node.feature]]
            preds.append(node.output)
        return np.array(preds)
    
    def evaluate(self, X, y ):
        preds = self.predict(X)
        return (y==preds).mean()
       
         


In [3]:
features = np.random.randint(size =(10,3), low=0, high=2)
labels = np.array([1,1,1,1,1,0,0,0,0,0])
features = np.array([
    [0,0,0,0,0,0,1,1,1,1],
    [0,0,0,0,0,1,1,1,1,0],
    [0,0,0,0,0,0,1,1,1,1],
    [0,1,0,0,1,1,0,1,1,1],
    [1,1,0,0,1,1,1,0,1,1]]).T
features, labels

(array([[0, 0, 0, 0, 1],
        [0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1],
        [0, 1, 0, 1, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1],
        [1, 0, 1, 1, 1]]),
 array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0]))

In [4]:
tree1 = BinaryTree()
tree1.train(X=features, y=labels)
tree1.pretty_print()
tree1.predict(features)

=> Depth : 0     |||     Entropy :  1.0   |||    Feature 0 |||   Num Features:  5
=======> Depth : 1     |||     Entropy :  0.6500224216483541   |||    Feature 1 |||   Num Features:  3
=============> Depth : 2     |||     Entropy :  0   |||    Confidence 1.0    |||    Output 1
=============> Depth : 2     |||     Entropy :  0   |||    Confidence 1.0    |||    Output 0
=======> Depth : 1     |||     Entropy :  0   |||    Confidence 1.0    |||    Output 0


array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

In [28]:
labels = np.array([1,1,1,1,1,0,0,0,1,0,0,1,1,0,1,0,1])
features = np.array([
    [0,0,1,0,0,0,1,1,0,1,1,0,1,0,0,1,1],
    [0,1,0,0,0,1,1,0,1,0,1,0,1,0,1,1,1],
    [1,0,1,1,0,0,1,1,1,1,1,0,0,0,0,1,0],
    [0,1,0,0,1,1,0,1,1,1,1,0,1,0,1,1,1],
    [1,1,0,0,1,1,1,0,1,1,1,0,1,0,1,0,1]]).T
samples = 50
features = 10
labels = np.random.randint(size=(samples,), low=0, high=2)
features = np.random.randint(size=(samples,features), low=0, high=2)
tree1 = BinaryTree(threshold=1,depth=4)
tree1.train(X=features, y=labels, verbose=True)
tree1.evaluate(features,labels)

Initializing Root
Getting Best feature
Retrieved best feature :  3
Setting Child 0 of Depth 1
Confidence 0.652174 Num Samples at Node : 23
Threshold Not met 
Getting Best feature
Retrieved best feature :  5
Setting Child 0 of Depth 2
Confidence 0.875000 Num Samples at Node : 8
Threshold Not met 
Getting Best feature
Retrieved best feature :  9
Setting Child 0 of Depth 3
Confidence 1.000000 Num Samples at Node : 5
Threshold Met 
Setting Child 1 of Depth 3
Confidence 0.666667 Num Samples at Node : 3
Threshold Not met 
Getting Best feature
Retrieved best feature :  1
Setting Child 0 of Depth 4
Confidence 1.000000 Num Samples at Node : 1
Threshold Met 
Setting Child 1 of Depth 4
Confidence 1.000000 Num Samples at Node : 2
Threshold Met 
Setting Child 1 of Depth 2
Confidence 0.533333 Num Samples at Node : 15
Threshold Not met 
Getting Best feature
Retrieved best feature :  6
Setting Child 0 of Depth 3
Confidence 0.600000 Num Samples at Node : 10
Threshold Not met 
Getting Best feature
Retri

0.88

In [27]:
tree1.pretty_print()

=> Depth : 0     |||     Entropy :  0.998846   |||    Feature 4 |||   Num Features:  5
=======> Depth : 1     |||     Entropy :  0.950956   |||    Feature 2 |||   Num Features:  4
=============> Depth : 2     |||     Entropy :  0.970951   |||    Feature 0 |||   Num Features:  3
===================> Depth : 3     |||     Entropy :  0.985228   |||    Feature 3 |||   Num Features:  2
=========================> Depth : 4     |||     Entropy :  1.000000   |||    Feature 1 |||   Num Features:  1
===============================> Depth : 5     |||     Entropy :  1.000000   |||    Confidence 0.500000    |||    Output 1
===============================> Depth : 5     |||     Entropy :  1.000000   |||    Confidence 0.500000    |||    Output 1
=========================> Depth : 4     |||     Entropy :  0.918296   |||    Confidence 0.666667    |||    Output 0
===================> Depth : 3     |||     Entropy :  0.811278   |||    Feature 1 |||   Num Features:  2
=========================> Depth : 4 

In [23]:
tree1.evaluate(features, labels)

1.0